In [1]:
#testing whether api keys are stored or not

import os
#print(os.environ["GROQ_API_KEY"])
#print(os.environ["PINECONE_API_KEY"])

In [ ]:
#Start executing code from below

In [1]:
# Groq LLM is used as OpenAI API is a paid service

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

loader = TextLoader('./data.txt')

#reading the content of file. It contains info on how July 2024 will be for people of Sagittarius zodiac sign.
#info is obtained from a Hindustan Times article
documents = loader.load() 

#text is split into smaller chunks of size 1000 characters each,
#context_overlap helps model in understanding context between chunks.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4) 
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings() #this function generates embeddings

C:\Users\Admin\AppData\Local\Temp\ipykernel_11228\978250695.py:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings() #this function generates embeddings
d:\Rohan\ML\Projects\rag_saggitarius\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\Rohan\ML\Projects\rag_saggitarius\venv\Lib\site-packages\transformers\utils\hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
d:\Rohan\ML\Projects\rag_saggitarius\venv\Lib\site-packages\transformers\tokenization_utils_base.py:160

In [4]:
documents[0].page_content

'This July, Sagittarius can expect positive changes in love, career, finances, and health. Your optimistic and adventurous spirit will guide you to success in your career endeavors this month. July brings an abundance of opportunities and optimism for Sagittarians. You will experience growth and positive changes in various aspects of your life, including love, career, money, and health. Overall, expect financial stability and growth throughout the month. This month, Sagittarius, your love life takes a delightful turn. If youâ€™re single, expect new romantic interests to emerge, possibly leading to a meaningful relationship. Those already in relationships will find deeper emotional connections with their partners. Communication is keyâ€”open up and express your feelings honestly. Be mindful of your partnerâ€™s needs and work towards mutual understanding. Sagittarians will find their career paths illuminated with new opportunities and possibilities this July. You may receive recognition 

In [6]:
from langchain.vectorstores import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"]) # an instance of pinecone is created
index_name = "rag-index" # we need to create a what is called index. Here the vectors are stored


if index_name not in pc.list_indexes().names():
  pc.create_index(name=index_name, metric="cosine", dimension=768, spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) )
  docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name) #docsearch stores the embeddings of the docs
else:
  docsearch = PineconeVectorStore.from_texts(texts=docs[0].page_content,embedding=embeddings, index_name=index_name) 
  # docs[0] might be wrong. documents[0] might be correct

In [7]:
from langchain import PromptTemplate

template = """
You are a fortune teller. This human will ask you questions about their life. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 2 sentences and concise.

Context: {context}
Question: {question}
Answer: 

"""
#this is the prompt which will help us in generating an answer
prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [8]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

#docsearch contains the context (info about July 2024 of Sagittarius people)
#RunnablePassThrough() passes the question to the next part of the RAG chain
#prompt puts the context and question into the required format
#llm generates response based on the prompt
#StrOutputParser() parses the LLM's response into a clean, formatted string
rag_chain = (
  {"context": docsearch.as_retriever(),  "question": RunnablePassthrough()} 
  | prompt 
  | llm
  | StrOutputParser() 
)

In [9]:
result = rag_chain.invoke("How will my July 2024 be?") #RAG process starts working here and a response is generated
print(result)

A look into your July 2024 reveals a busy and exciting period ahead! Expect a surge in your professional life, with new opportunities and challenges that will help you grow and learn.
